In [ ]:
import pandas as pd

In [ ]:
buzzfeed_df = pd.read_csv('data/buzzfeed.csv')
isot_df = pd.read_csv('data/isot.csv')

In [ ]:
combined_df = pd.concat([buzzfeed_df, isot_df], ignore_index=True)

In [ ]:
combined_df['label'] = combined_df['label'].map({'fake': 0, 'real': 1})

In [ ]:
from transformers import RobertaTokenizer

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [ ]:
def tokenize(text):
    return tokenizer(text, padding='max_length', truncation=True, max_length=512, return_tensors="pt")


In [ ]:
tokenized_texts = combined_df['text'].apply(tokenize)

In [ ]:
input_ids = torch.stack([item['input_ids'].squeeze() for item in tokenized_texts])
attention_masks = torch.stack([item['attention_mask'].squeeze() for item in tokenized_texts])
labels = torch.tensor(combined_df['label'].values)

In [ ]:
import torch
from torch import nn
from transformers import RobertaModel

In [ ]:
class RoBERTaForFakeNewsDetection(nn.Module):
    def __init__(self):
        super(RoBERTaForFakeNewsDetection, self).__init__()
        self.roberta = RobertaModel.from_pretrained("roberta-base")
        self.classifier = nn.Linear(self.roberta.config.hidden_size, 2)

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.classifier(outputs.last_hidden_state[:, 0, :])
        return logits

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
from transformers import AdamW

In [ ]:
dataset = TensorDataset(input_ids, attention_masks, labels)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
model = RoBERTaForFakeNewsDetection()
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=1e-2)

In [ ]:
for epoch in range(50):
    model.train()
    for batch in dataloader:
        optimizer.zero_grad()
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch+1}, Loss: {loss.item()}")